In [1]:
# load the packages needed to run the code
import os
import pandas as pd
import geopandas as gpd
import numpy as np
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import re
from webdriver_manager.chrome import ChromeDriverManager
from webdriver_manager.utils import ChromeType
#from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm

In [2]:
activity_list = ['Backpacking',
                 'Bird watching',
                 'Camping',
                 'Cross country skiing',
                 'Fishing',
                 'Hiking',
                 'Horseback riding',
                 'Mountain biking',
                 'Nature trips',
                 'Off-road driving',
                 'Paddle sports',
                 'Road biking',
                 'Rock climbing',
                 'Scenic driving',
                 'Skiing',
                 'Snowshoeing',
                 'Surfing',
                 'Running',
                 'Walking']

attraction_list = ['Beach',
                   'Cave',
                   'City walk',
                   'Forest',
                   'Historic site',
                   'Hot springs',
                   'Lake',
                   'Pub walk',
                   'Rails trails',
                   'River',
                   'Views',
                   'Waterfall',
                   'Wild flowers',
                   'Wildlife']

suitability_list = ['Dog friendly',
                    'Kid friendly',
                    'Partially paved',
                    'Paved',
                    'Stroller friendly',
                    'Wheelchair friendly']

In [3]:
# specify the path where you want to save the .csv file
#outpath = '/Users/spk/Documents/Code/Python/webscrape_proj/outputs/banff_national_park_trailforks.csv'
outDIR = '/Users/spk/Documents/GitHub/trail_scraping/outputs/'
outDIR_csvs = '/Users/spk/Documents/GitHub/trail_scraping/outputs/AllTrails_csv/'
if not os.path.exists(outDIR):
    os.mkdir(outDIR)
out_name = 'AllTrails_UC_middle_watersheds.csv'
# specify the url that you want to scrape from

In [5]:
inSHP_f = '/Users/spk/Documents/Projects/Y2Y/Data/UC_middle_area/UC_middle_watersheds.shp'

roi = gpd.read_file(inSHP_f)

In [6]:
#roi_bounds = roi.to_crs('EPSG:4326').total_bounds
#br_lat_list = np.arange(float(roi_bounds.miny), float(roi_bounds.maxy), 0.20)
#tl_lng_list = np.arange(float(roi_bounds.minx), float(roi_bounds.maxx), 0.40)
roi_bounds = roi.to_crs('EPSG:4326').total_bounds
br_lat_list = np.arange(roi_bounds[1], roi_bounds[3], 0.20)
tl_lng_list = np.arange(roi_bounds[0], roi_bounds[2], 0.40)

In [7]:
options = webdriver.ChromeOptions()
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument('--no-sandbox') 
options.add_argument('--disable-dev-shm-usage')
options.add_argument('window-size=1920,1080')

driver = webdriver.Chrome(ChromeDriverManager(chrome_type=ChromeType.GOOGLE).install(),
                         options=options)
#driver.maximize_window()
#driver.execute_script("document.body.style.zoom='50%'")
timeout = 15

[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/89.0.4389.23/chromedriver_mac64.zip


[WDM] - Driver has been saved in cache [/Users/spk/.wdm/drivers/chromedriver/mac64/89.0.4389.23]


In [8]:
df_out_all = pd.DataFrame(columns=['Name', 'URL', 'Zone', 'Difficulty', 'Rating', 'Reviews'])

for lat in br_lat_list:
    for lng in tl_lng_list:
        b_tl_lat = lat + 0.20
        b_tl_lng = lng
        b_br_lat = lat
        b_br_lng = lng + 0.40
        
        if os.path.exists(outDIR_csvs + 'AllTrails_basic_' + str(b_tl_lat) + str(b_tl_lng) + '.csv'):
            print('File already exists. Appending and moving to next map bounds.')
            df_out = pd.read_csv(outDIR_csvs + 'AllTrails_basic_' + str(b_tl_lat) + str(b_tl_lng) + '.csv')
            df_out_all = df_out_all.append(df_out)
            continue
        else:
            for attempt in range(10):
                try:
                    print('\nSearching map bounds: (' + str(b_tl_lat) + ', ' +
                         str(b_tl_lng) + ', ' + str(b_br_lat) + ', ' + str(b_br_lng) + ')')

                    url_main = 'https://www.alltrails.com/explore?b_tl_lat=' + str(b_tl_lat) + \
                    '&b_tl_lng=' + str(b_tl_lng) + \
                    '&b_br_lat=' + str(b_br_lat) + \
                    '&b_br_lng=' + str(b_br_lng)
                    driver.get(url_main)

                    trail_num_xp = '/html/body/div[3]/div[2]/main/div/div/div/div[3]/div[1]/div/div[1]'
                    trail_table_elem = WebDriverWait(driver, timeout)\
                        .until(ec.presence_of_element_located((By.XPATH, 
                                                               trail_num_xp)))

                    trail_num_txt = trail_table_elem.find_element(By.XPATH,
                                                     trail_num_xp).text
                    trail_num_str = trail_num_txt[trail_num_txt.find("(")+1:trail_num_txt.find(")")]
                    print(trail_num_str + ' trails found.')


                    if trail_num_str == '1000+':
                        print('Too many trails, need to zoom in on map')
                    else:
                        trail_num = int(trail_num_str)


                    scrollPane = trail_table_elem.find_element(By.XPATH, 
                                                       '/html/body/div[3]/div[2]/main/div/div/div/div[3]/div[1]/ul')


                    viz_trails = scrollPane.find_elements(By.XPATH,
                                                  '//*[@id="fullscreen-search-results"]/ul/li')
                    trail_num_viz = len(viz_trails)

                    ActionChains(driver).move_to_element_with_offset(scrollPane,
                                                            xoffset=5,
                                                            yoffset=5).click().perform()

                    while trail_num_viz < trail_num:
                        ActionChains(driver).send_keys(Keys.PAGE_DOWN).perform()
                        viz_trails = scrollPane.find_elements(By.XPATH,
                                                          '//*[@id="fullscreen-search-results"]/ul/li')
                        trail_num_viz = len(viz_trails)
                        print('\r' + str(trail_num_viz), end='\r', flush=True)

                    if trail_num_viz > trail_num:
                        viz_trails = viz_trails[:trail_num]
                    print('Getting trail links and primary data.')
                    df_out = pd.DataFrame(columns=['Name', 'URL', 'Zone', 'Difficulty', 'Rating', 'Reviews'])
                    for idx, t in enumerate(viz_trails):
                        name = t.find_element(By.XPATH,
                                              './/div/a').get_attribute('text')
                        print_space = ' ' * (70-len(name))
                        print('\r' + str(idx+1) + ' of ' + str(trail_num) + ': ' + name + print_space, 
                              end='', flush=True)
                        url = t.find_element(By.XPATH,
                                          './/div/a').get_attribute('href')
                        zone = t.find_element(By.XPATH,
                                          './/div/div[2]/a').get_attribute('text')  
                        diff = t.find_element(By.XPATH,
                                          './/div[2]/div[2]/span[1]').text 
                        try:
                            rating = re.sub(' Stars', '', t.find_element(By.XPATH,
                                                                     './/div[2]/div[2]/span[2]').get_attribute('aria-label'))
                        except NoSuchElementException:
                            rating = 'NA'
                        try:
                            revs = int(t.find_element(By.XPATH,
                                                      './/div[2]/div[2]/span[3]').text.replace('(', '').replace(')', ''))
                        except NoSuchElementException:
                            revs = 0
                        df_tmp = pd.DataFrame({
                            'Name': name,
                            'URL': url,
                            'Zone': zone,
                            'Difficulty': diff,
                            'Rating': rating,
                            'Reviews': revs
                        }, index=[len(df_out.index)])
                        df_out = df_out.append(df_tmp)

                    df_out.to_csv(outDIR_csvs + 
                                  'AllTrails_basic_' + str(b_tl_lat) + str(b_tl_lng) + '.csv',
                                 index=False)
                    df_out_all = df_out_all.append(df_out)
                except (TimeoutException, TimeoutError) as e:
                    if attempt < 9:
                        print('Timeout error. Retrying.')
                        continue
                    else:
                        raise
                else:
                    break
            else:
                print('ERROR: Process failed after 10 retries.')
                    
                    
df_out_all.to_csv(outDIR + 'AllTrails_basic_' + re.sub('.shp', '.csv', 
                                                       inSHP_f.split('/')[-1]),
                 index=False)

File already exists. Appending and moving to next map bounds.
File already exists. Appending and moving to next map bounds.
File already exists. Appending and moving to next map bounds.
File already exists. Appending and moving to next map bounds.
File already exists. Appending and moving to next map bounds.
File already exists. Appending and moving to next map bounds.
File already exists. Appending and moving to next map bounds.
File already exists. Appending and moving to next map bounds.
File already exists. Appending and moving to next map bounds.
File already exists. Appending and moving to next map bounds.
File already exists. Appending and moving to next map bounds.
File already exists. Appending and moving to next map bounds.
File already exists. Appending and moving to next map bounds.
File already exists. Appending and moving to next map bounds.
File already exists. Appending and moving to next map bounds.
File already exists. Appending and moving to next map bounds.
File alr

In [9]:
#basic_csvs = [f for f in os.listdir(outDIR_csvs) if re.match('.*_basic.*csv', f)]

In [53]:
#outDIR_csvs + 'AllTrails_details_' + str_b_tl_lat + str_b_tl_lng + '.csv'

'/Users/spk/Documents/GitHub/trail_scraping/outputs/AllTrails_csv/AllTrails_details_50.66284241277868-116.59250188825408.csv'

In [56]:
df_out_fnl = pd.read_csv(outDIR + 'AllTrails_basic_' + re.sub('.shp', '.csv', 
                                                              inSHP_f.split('/')[-1]))

options = webdriver.ChromeOptions()
options = webdriver.ChromeOptions()
options.add_argument("--headless")
options.add_argument('--no-sandbox') 
options.add_argument('--disable-dev-shm-usage')
options.add_argument('window-size=1920,1080')

driver = webdriver.Chrome(ChromeDriverManager(chrome_type=ChromeType.GOOGLE).install(),
                         options=options)
#driver.maximize_window()
#driver.execute_script("document.body.style.zoom='50%'")
timeout = 15
new_cols = ['Elevation_gain_m',
            'Traffic',
            'Activities',
            'Attractions',
            'Suitabilities']

print('\nGetting individual trail details.')
idx_all = 0
for lat in br_lat_list:
    for lng in tl_lng_list:
        str_b_tl_lat = str(lat + 0.20)
        str_b_tl_lng = str(lng)
        print(str_b_tl_lat + ', ' + str_b_tl_lng)
        #str_b_tl_lat = f.split('_')[-1].split('-')[0]
        #str_b_tl_lng = '-' + re.sub('.csv', '', f.split('_')[-1].split('-')[-1])
        if os.path.exists(outDIR_csvs + 'AllTrails_details_' + str_b_tl_lat + 
                          str_b_tl_lng + '.csv'):
            print('File already exists. Appending and moving to next map bounds.')
            df_out = pd.read_csv(outDIR_csvs + 'AllTrails_details_' + str_b_tl_lat + 
                                 str_b_tl_lng + '.csv')
            if len(df_out) == 0:
                continue
            else:
                df_out_fnl.loc[idx_all:idx_all+len(df_out.index), new_cols] = df_out.loc[:, new_cols]
                idx_all += len(df_out)
                continue
        else:
            df_out = pd.read_csv(os.path.join(outDIR_csvs, 
                                             'AllTrails_basic_' + str_b_tl_lat + 
                                              str_b_tl_lng + '.csv'))
            for idx, url in enumerate(df_out['URL']):
                idx_all += 1
                for attempt in range(10):
                    try:
                        name = df_out.loc[idx, 'Name']
                        print_space = ' ' * (70-len(name))
                        print('\x1b[1K\r' + str(idx_all) + ' of ' + str(len(df_out_fnl)) + ': ' + name + print_space, 
                          end='\x1b[1K\r', flush=True)
                        driver.get(re.sub('\?ref=result-card', '', re.sub('/explore', '', url)))
                        elev_gain = WebDriverWait(driver, timeout)\
                        .until(ec.presence_of_element_located(
                            (By.XPATH,
                             '//*[@id="main"]/div[2]/div[1]/article/section[2]/div/span[2]/span[2]'))).text
                        df_out.loc[idx, 'Route type'] = driver.find_element(
                            By.XPATH,
                            '//*[@id="main"]/div[2]/div[1]/article/section[2]/div/span[3]/span[2]').text
                        if bool(re.search(' ft', elev_gain)):
                            df_out.loc[idx, 'Elevation_gain_m']  = round(
                                float(re.sub(' ft', '', elev_gain).replace(',', '')) * 0.3048, 2)
                        else:
                            df_out.loc[idx, 'Elevation_gain_m']  = round(
                                float(re.sub(' m', '', elev_gain).replace(',', '')) * 0.3048, 2)
                        para = driver.find_element(By.XPATH,
                                              '//*[@id="auto-overview"]').text.split()
                        try:
                            df_out.loc[idx, 'Traffic'] = para[para.index('trafficked') - 1]
                        except ValueError:
                            df_out.loc[idx, 'Traffic'] = 'NA'
                        tag_cloud = driver.find_elements(By.XPATH, 
                                                         '//*[@id="main"]/div[2]/div[1]/article/section[3]/*')
                        activities = []
                        attractions = []
                        suitabilities = []
                        for tag in tag_cloud:
                            if tag.text in activity_list:
                                activities.append(tag.text)
                            if tag.text in attraction_list:
                                attractions.append(tag.text)
                            if tag.text in suitability_list:
                                suitabilities.append(tag.text)
                        df_out.loc[idx, 'Activities'] = ', '.join(activities)
                        df_out.loc[idx, 'Attractions'] = ', '.join(attractions)
                        df_out.loc[idx, 'Suitabilities'] = ', '.join(suitabilities)
                        df_out.to_csv('/Users/spk/Documents/GitHub/trail_scraping/outputs/AllTrails_csv/' +
                                      'AllTrails_details_' + str_b_tl_lat + str_b_tl_lng + '.csv')

                        df_out_fnl.loc[idx_all-1, new_cols] = df_out.loc[idx, new_cols]
                    except (TimeoutException, TimeoutError) as e:
                        if attempt < 9:
                            continue
                        else:
                            raise
                    else:
                        break

                else:
                    print('Process failed after 10 retries')

df_out_fnl = df_out_fnl.drop_duplicates()
df_out_fnl.to_csv(outDIR + 'AllTrails_details_' + re.sub('.shp', '.csv', 
                                                         inSHP_f.split('/')[-1]))

[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [/Users/spk/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


 

Getting individual trail details.
50.262842412778674, -118.99250188825411
50.262842412778674, -118.5925018882541                                          
50.262842412778674, -118.1925018882541shee Creek Trail                          
50.262842412778674, -117.7925018882541                                          
50.262842412778674, -117.39250188825409                                          
50.262842412778674, -116.99250188825408                                          
50.262842412778674, -116.59250188825408                                          
50.46284241277868, -118.99250188825411
50.46284241277868, -118.5925018882541
50.46284241277868, -118.1925018882541                                            
50.46284241277868, -117.7925018882541                                            
50.46284241277868, -117.39250188825409                                           
50.46284241277868, -116.99250188825408
50.46284241277868, -116.59250188825408                                  

In [57]:
#df_out_fnl
df_out_fnl_bin = pd.concat([df_out_fnl,
                            df_out_fnl.loc[:,'Activities'].str.replace(' ','').str.get_dummies(sep=','),
                            df_out_fnl.loc[:, 'Attractions'].str.replace(' ', '').str.get_dummies(sep=','),
                            df_out_fnl.loc[:, 'Suitabilities'].str.replace(' ', '').str.get_dummies(sep=',')
                           ], axis=1)

In [58]:
df_out_fnl_bin.to_csv(outDIR + 'AllTrails_details_binary' + re.sub('.shp', '.csv',
                                                                   inSHP_f.split('/')[-1]))

112 of 112: Mount Glasgow and Mount Cornwall via Little Elbow Trail                                                                 